<a href="https://colab.research.google.com/github/ariel5914/Python-perfume-recommendation/blob/main/%EA%B4%80%EC%95%85%EA%B5%AC_%EC%A0%90%EC%88%98%EB%B6%80%EC%97%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

def mercator_projection(lat, lon):
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

def count_objects_within_radius(center_lat, center_lon, neighborhoods, radius):
    center = mercator_projection(center_lat, center_lon)
    total_rate = 0.0
    count = 0
    for neighborhood, rate in neighborhoods.items():
        neighborhood_cartesian = mercator_projection(neighborhood[0], neighborhood[1])
        distance = math.sqrt((neighborhood_cartesian[0]-center[0])**2 + (neighborhood_cartesian[1]-center[1])**2)
        if distance <= radius:
            total_rate += rate
            count += 1
    if count > 0:
        avg_rate = total_rate / count
    else:
        avg_rate = 0.0
    return avg_rate, count

In [ ]:
center_lat = 37.7749
center_lon = -122.4194
neighborhoods = {(37.7749, -122.4150): 0.1, (37.7730, -122.4216): 0.3,
                 (37.7724, -122.4168): 0.2, (37.7714, -122.4174): 0.4, (37.7784, -122.4192): 0.5}
radius = 500

avg_rate, count = count_objects_within_radius(center_lat, center_lon, neighborhoods, radius)
print("Average rate within radius:", avg_rate)
print("Number of neighborhoods within radius:", count)

Average rate within radius: 0.275
Number of neighborhoods within radius: 4


### ''여러개''의 포인트가 있을 때 반경 내의 물체 세고, percentage 할당하는 코드

In [ ]:
import pandas as pd
import numpy as np
import pyproj
import folium
import json
import requests

embell = pd.read_excel('안심벨 데이터.xlsx',usecols=['위도', '경도'])
embell

x4 = pd.read_csv("서울시 유흥주점영업 인허가 정보.csv", encoding='EUC-KR',
                   usecols=['영업상태명','좌표정보(X)','좌표정보(Y)'])
x4['좌표정보(X)'] = pd.to_numeric(x4['좌표정보(X)'], errors="coerce")
x4['좌표정보(Y)'] = pd.to_numeric(x4['좌표정보(Y)'], errors="coerce")
x4

,영업상태명,좌표정보(X),좌표정보(Y)
0,영업/정상,203330.125000,444424.745000
1,영업/정상,210491.321884,443569.763243
2,폐업,NaN,NaN
3,영업/정상,193084.355458,445642.841120
4,영업/정상,201601.185141,446044.757176
...,...,...,...
4900,영업/정상,201630.614056,445872.941217
4901,영업/정상,193745.643373,446380.254633
4902,폐업,212067.798279,448341.061540
4903,폐업,212067.798279,448341.061540


In [ ]:
x5 = pd.read_csv("서울시 숙박업 인허가 정보.csv", encoding='EUC-KR',
                   usecols=['영업상태명','좌표정보(X)','좌표정보(Y)'])
x5['좌표정보(X)'] = pd.to_numeric(x5['좌표정보(X)'], errors="coerce")
x5['좌표정보(Y)'] = pd.to_numeric(x5['좌표정보(Y)'], errors="coerce")
x5

,영업상태명,좌표정보(X),좌표정보(Y)
0,영업/정상,199345.466097,451233.369781
1,영업/정상,198591.133748,450740.353160
2,영업/정상,198037.338494,451010.546630
3,영업/정상,198602.119250,450758.364667
4,영업/정상,198607.670947,450770.829936
...,...,...,...
6659,영업/정상,200161.563979,452132.172041
6660,영업/정상,186372.412823,449366.415349
6661,영업/정상,191832.871758,442881.349005
6662,영업/정상,200850.193897,456070.122190


In [ ]:
for i in range(len(x4)):

    if x4['영업상태명'][i] == '폐업':
        x4 = x4.drop(i,axis=0)
x4.index = range(len(x4))
x4

,영업상태명,좌표정보(X),좌표정보(Y)
0,영업/정상,203330.125000,444424.745000
1,영업/정상,210491.321884,443569.763243
2,영업/정상,193084.355458,445642.841120
3,영업/정상,201601.185141,446044.757176
4,영업/정상,206020.863854,451931.503922
...,...,...,...
1784,영업/정상,191422.697775,437647.525127
1785,영업/정상,190771.595590,440515.669035
1786,영업/정상,191722.228678,446262.797865
1787,영업/정상,201630.614056,445872.941217


In [ ]:
for i in range(len(x5)):

    if x5['영업상태명'][i] == '폐업':
        x5 = x5.drop(i,axis=0)
x5.index = range(len(x5))
x5

,영업상태명,좌표정보(X),좌표정보(Y)
0,영업/정상,199345.466097,451233.369781
1,영업/정상,198591.133748,450740.353160
2,영업/정상,198037.338494,451010.546630
3,영업/정상,198602.119250,450758.364667
4,영업/정상,198607.670947,450770.829936
...,...,...,...
2488,영업/정상,200161.563979,452132.172041
2489,영업/정상,186372.412823,449366.415349
2490,영업/정상,191832.871758,442881.349005
2491,영업/정상,200850.193897,456070.122190


In [ ]:
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """

    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:,0], coord[:,1])
    return np.dstack([fx, fy])[0]

In [ ]:
x4 = x4.drop('영업상태명',axis=1)
coord = np.array(x4)
p1_type="epsg:2097"
p2_type="epsg:4326"

result = project_array(coord, p1_type, p2_type)
x4['경도'] = result[:,0]
x4['위도'] = result[:,1]
x4

c:\Users\user\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\user\AppData\Local\Temp\ipykernel_9056\4036950851.py:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.

,좌표정보(X),좌표정보(Y),경도,위도
0,203330.125000,444424.745000,127.035559,37.502029
1,210491.321884,443569.763243,127.116534,37.494273
2,193084.355458,445642.841120,126.919677,37.512984
3,201601.185141,446044.757176,127.016010,37.516630
4,206020.863854,451931.503922,127.066050,37.569651
...,...,...,...,...
1784,191422.697775,437647.525127,126.900976,37.440932
1785,190771.595590,440515.669035,126.893582,37.466768
1786,191722.228678,446262.797865,126.904263,37.518558
1787,201630.614056,445872.941217,127.016342,37.515082


In [ ]:
x5 = x5.drop('영업상태명',axis=1)
coord = np.array(x5)
p1_type="epsg:2097"
p2_type="epsg:4326"

result = project_array(coord, p1_type, p2_type)
x5['경도'] = result[:,0]
x5['위도'] = result[:,1]
x5

c:\Users\user\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\user\AppData\Local\Temp\ipykernel_9056\4036950851.py:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.

,좌표정보(X),좌표정보(Y),경도,위도
0,199345.466097,451233.369781,126.990490,37.563380
1,198591.133748,450740.353160,126.981953,37.558937
2,198037.338494,451010.546630,126.975684,37.561371
3,198602.119250,450758.364667,126.982077,37.559100
4,198607.670947,450770.829936,126.982140,37.559212
...,...,...,...,...
2488,200161.563979,452132.172041,126.999727,37.571479
2489,186372.412823,449366.415349,126.843690,37.546458
2490,191832.871758,442881.349005,126.905552,37.488093
2491,200850.193897,456070.122190,127.007526,37.606959


In [ ]:
def compute_center_point_rates(center_points, neighborhoods, radius):
    center_rates = []
    for center_lat, center_lon in center_points:
        avg_rate, count = count_objects_within_radius(center_lat, center_lon, neighborhoods, radius)
        center_rates.append((center_lat, center_lon, avg_rate, count))
    return center_rates

In [ ]:
center_points = []
for i in range(len(embell)):
    center_points.append((embell['위도'][i], embell['경도'][i]))

neighborhoods = {}
for i in range(len(x4)):
    neighborhoods[(x4['위도'][i], x4['경도'][i])] = 0.237

for i in range(len(x5)):
    neighborhoods[(x5['위도'][i], x5['경도'][i])] = 0.287

radius = 500


center_rates = compute_center_point_rates(center_points, neighborhoods, radius)
for center_rate in center_rates:
    print("Center point: ({:.4f}, {:.4f}), Average rate within radius: {:.2f}, Number of neighborhoods within radius: {}"
          .format(center_rate[0], center_rate[1], center_rate[2], center_rate[3]))


Center point: (37.4757, 126.9707), Average rate within radius: 0.00, Number of neighborhoods within radius: 0
Center point: (37.4680, 126.9455), Average rate within radius: 0.00, Number of neighborhoods within radius: 0
Center point: (37.4683, 126.9455), Average rate within radius: 0.00, Number of neighborhoods within radius: 0
Center point: (37.4796, 126.9375), Average rate within radius: 0.24, Number of neighborhoods within radius: 1
Center point: (37.4753, 126.9764), Average rate within radius: 0.26, Number of neighborhoods within radius: 28
Center point: (37.4833, 126.9597), Average rate within radius: 0.00, Number of neighborhoods within radius: 0
Center point: (37.4765, 126.9677), Average rate within radius: 0.00, Number of neighborhoods within radius: 0
Center point: (37.4773, 126.9637), Average rate within radius: 0.00, Number of neighborhoods within radius: 0
Center point: (37.4830, 126.9588), Average rate within radius: 0.00, Number of neighborhoods within radius: 0
Center po

In [ ]:
center_rates

[(37.47567532, 126.9707322, 0.0, 0),
 (37.46802993, 126.945548, 0.0, 0),
 (37.46832312, 126.9454603, 0.0, 0),
 (37.47959612, 126.9375368, 0.237, 1),
 (37.47525139, 126.9764479, 0.25842857142857145, 28),
 (37.48325208, 126.9597224, 0.0, 0),
 (37.47652335, 126.9676827, 0.0, 0),
 (37.47725749, 126.9636574, 0.0, 0),
 (37.4829696, 126.9588399, 0.0, 0),
 (37.47914921, 126.9618488, 0.0, 0),
 (37.47614513, 126.9691611, 0.0, 0),
 (37.46556777, 126.9331702, 0.0, 0),
 (37.46469588, 126.9315369, 0.0, 0),
 (37.46100702, 126.9322949, 0.0, 0),
 (37.47144923, 126.9312522, 0.287, 5),
 (37.47627005, 126.9396876, 0.0, 0),
 (37.4769122, 126.9355224, 0.0, 0),
 (37.47598183, 126.9387483, 0.0, 0),
 (37.47657627, 126.9377913, 0.0, 0),
 (37.47462974, 126.9418547, 0.0, 0),
 (37.47749692, 126.9328241, 0.0, 0),
 (37.47632048, 126.9346501, 0.0, 0),
 (37.47743135, 126.9312079, 0.0, 0),
 (37.479091, 126.9324208, 0.0, 0),
 (37.49182, 126.9469266, 0.0, 0),
 (37.48698336, 126.9560607, 0.287, 1),
 (37.48821956, 126.9551

In [ ]:
# !pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="9812jy@gmail.com")


#geolocator = Nominatim(user_agent="geoapiExercises")

def get_location_by_coordinates(latitude, longitude):
    location = geolocator.reverse(str(latitude) + ", " + str(longitude))
    return location

latitude = 37.478109
longitude = 126.951492
location = get_location_by_coordinates(latitude, longitude)
print(location.address)

관악구청, 쑥고개로, 관악구종합청사, 청룡동, 관악구, 서울, 08832, 대한민국


In [ ]:
# !pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="9812jy@gmail.com")


#geolocator = Nominatim(user_agent="geoapiExercises")

def get_location_by_coordinates(latitude, longitude):
    location = geolocator.reverse(str(latitude) + ", " + str(longitude))
    return location

score={}
for center_rate in center_rates:
    latitude = center_rate[0]
    longitude = center_rate[1]

    score[(latitude, longitude)] = pd.to_numeric(center_rate[2])
    location = get_location_by_coordinates(latitude, longitude)
    if '보라매동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 3.493

    elif '은천동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 1.913

    elif '성현동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 2.630

    elif '중앙동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 3.044

    elif '청림동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 3.543

    elif '행운동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 2.962

    elif '청룡동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 2.101

    elif '낙성대동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 3.751

    elif '인헌동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 4.118

    elif '남현동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 4.743

    elif '신림동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 2.231

    elif '신사동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 2.953

    elif '조원동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 3.640

    elif '미성동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 3.691

    elif '난곡동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 3.841

    elif '난향동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 6.492

    elif '서원동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 2.036

    elif '신원동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 4.014

    elif '서림동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 2.351

    elif '삼성동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 6.180

    elif '대학동' in str(location):
        score[(latitude, longitude)] = score[(latitude, longitude)] + 2.569

    else:
        print(latitude, longitude, location, '정보 없음')

37.49299991 126.9342896 국사봉10길, 상도3동, 동작구, 서울, 07043, 대한민국 정보 없음


In [ ]:
score

{(37.46347, 126.91943): 6.492}